In [ ]:
# %% [markdown]
# Football Analysis Colab Worker
# 
# **Steps to Run:**
# 1. Click "Runtime" -> "Change runtime type" -> Select "GPU"
# 2. Run all cells (Ctrl+F9)
# 3. Keep tab open for processing

# %% [code]
# Install dependencies
!pip install ultralytics opencv-python-headless deep-sort-realtime google-generativeai requests matplotlib pandas numpy -q

# %% [code]
import os
import cv2
import time
import requests
import numpy as np
import pandas as pd
from google.colab import files
from IPython.display import clear_output
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import google.generativeai as genai

# Configuration
CONFIG = {
    "api_key": "your-secret-key",          # Match with backend
    "backend_url": "https://your-render-app.onrender.com",
    "frame_skip": 5,                       # Process every Nth frame
    "resize_width": 640,                    # Processing resolution
    "possession_threshold": 0.05,          # 5% of frame width
    "max_retries": 5,
    "poll_interval": 15                    # Seconds between checks
}

# Initialize models
genai.configure(api_key=CONFIG["api_key"])
gemini_model = genai.GenerativeModel('gemini-pro')
yolo_model = YOLO('yolov8n.pt')
tracker = DeepSort(max_age=30)

# %% [code]
def process_video(job_id, video_url):
    """Main video processing pipeline"""
    try:
        print(f"Starting processing for job {job_id}")
        
        # Download video
        video_path = f"temp_{job_id}.mp4"
        download_video(video_url, video_path)
        
        # Process video
        players, ball_data, fps = track_players(video_path)
        stats_df = calculate_stats(players, ball_data, fps)
        results = generate_results(stats_df)
        
        # Cleanup
        os.remove(video_path)
        return {
            "status": "completed",
            "results": results,
            "players": len(stats_df)
        }
    except Exception as e:
        return {
            "status": "failed",
            "error": str(e)
        }

def download_video(url, save_path):
    """Download video from URL"""
    response = requests.get(url, stream=True)
    with open(save_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# %% [code]
def track_players(video_path):
    """Video processing with YOLO + DeepSORT"""
    cap = cv2.VideoCapture(video_path)
    players = {}
    ball_positions = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % CONFIG["frame_skip"] != 0:
            continue

        # Resize frame
        frame = cv2.resize(frame, (CONFIG["resize_width"], 
                                 int(CONFIG["resize_width"]*(9/16))))
        
        # Detect objects
        results = yolo_model.predict(frame, classes=[0,1,2], conf=0.3, verbose=False)
        
        # Track ball and players
        current_ball_pos = track_ball(results, frame)
        tracks = track_players_frame(results, frame)
        
        # Update player stats
        update_player_data(players, tracks, current_ball_pos, frame.shape)

    cap.release()
    return players, ball_positions, cap.get(cv2.CAP_PROP_FPS)

# %% [code]
def track_ball(results, frame):
    """Track ball position in current frame"""
    ball_boxes = [box for box, cls in zip(results[0].boxes.xyxy.cpu().numpy(),
                                        results[0].boxes.cls.cpu().numpy().astype(int))
                 if cls == 0 and results[0].boxes.conf.cpu().numpy()[0] > 0.25]
    
    if ball_boxes:
        avg_ball = np.mean(ball_boxes, axis=0)
        return ((avg_ball[0]+avg_ball[2])/2, (avg_ball[1]+avg_ball[3])/2)
    return None

def track_players_frame(results, frame):
    """Track players in current frame"""
    player_detections = [(box, conf, cls) for box, conf, cls in 
                        zip(results[0].boxes.xyxy.cpu().numpy(),
                            results[0].boxes.conf.cpu().numpy(),
                            results[0].boxes.cls.cpu().numpy().astype(int))
                        if cls in [1, 2]]  # 1=keeper, 2=player
    
    return tracker.update_tracks(player_detections, frame=frame)

# %% [code]
def update_player_data(players, tracks, ball_pos, frame_shape):
    """Update player statistics"""
    for track in tracks:
        if not track.is_confirmed():
            continue

        tid = int(track.track_id)
        x1, y1, x2, y2 = track.to_ltrb()
        pos = ((x1+x2)/2, (y1+y2)/2)

        if tid not in players:
            players[tid] = {
                'positions': [],
                'attack_time': 0,
                'defense_time': 0,
                'possession': 0,
                'speeds': [],
                'is_gk': track.det_class == 1
            }

        # Zone analysis
        if pos[0] > frame_shape[1] * 0.7:
            players[tid]['attack_time'] += 1
        elif pos[0] < frame_shape[1] * 0.3:
            players[tid]['defense_time'] += 1

        # Possession check
        if ball_pos:
            dist = np.hypot(pos[0]-ball_pos[0], pos[1]-ball_pos[1])
            if dist < frame_shape[1] * CONFIG["possession_threshold"]:
                players[tid]['possession'] += 1

        # Speed calculation (simplified)
        if players[tid]['positions']:
            prev_pos = players[tid]['positions'][-1]
            dx = (pos[0] - prev_pos[0]) * 0.1  # 0.1m/pixel assumption
            dy = (pos[1] - prev_pos[1]) * 0.1
            speed = np.hypot(dx, dy) * 3.6  # Convert to km/h
            players[tid]['speeds'].append(speed)

        players[tid]['positions'].append(pos)

# %% [code]
def calculate_stats(players, ball_data, fps):
    """Calculate final statistics"""
    stats = []
    total_frames = len(players[next(iter(players))]['positions']) if players else 1
    
    for pid, data in players.items():
        total = len(data['positions'])
        if total == 0:
            continue

        attack = min(data['attack_time']/total*10, 10)
        defense = min(data['defense_time']/total*10, 10)
        possession = (data['possession'] / total_frames) * 100
        max_speed = max(data['speeds'], default=0)
        position = "Goalkeeper" if data['is_gk'] else \
                  "Defender" if defense > 6 else \
                  "Midfielder" if (attack + defense) > 10 else "Attacker"

        stats.append({
            'player_id': pid,
            'position': position,
            'attack': round(attack, 1),
            'defense': round(defense, 1),
            'possession': round(possession, 1),
            'max_speed': round(max_speed, 1)
        })
    
    return pd.DataFrame(stats)

# %% [code]
def generate_results(df):
    """Generate analysis and visualizations"""
    results = []
    for _, row in df.iterrows():
        # Gemini analysis
        analysis = gemini_model.generate_content(
            f"Analyze football player stats: {row.to_dict()}"
        ).text
        
        # Radar chart
        chart_path = f"radar_{row['player_id']}.png"
        create_radar_chart(row, chart_path)
        
        results.append({
            'player_id': row['player_id'],
            'analysis': analysis,
            'chart_url': upload_to_host(chart_path)
        })
    return results

def create_radar_chart(row, save_path):
    """Generate normalized radar chart"""
    categories = ['Attack', 'Defense', 'Possession', 'Speed']
    values = [
        row['attack'],
        row['defense'],
        row['possession'] / 10,
        row['max_speed'] / 4
    ]
    
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, polar=True)
    ax.plot(angles, values, 'b-')
    ax.fill(angles, values, 'b', alpha=0.25)
    ax.set_xticks(angles)
    ax.set_xticklabels(categories)
    plt.savefig(save_path)
    plt.close()

def upload_to_host(file_path):
    """Mock upload function - replace with actual upload logic"""
    return f"https://storage.example.com/{file_path}"

# %% [code]
# Main worker loop
print("🚀 Starting football analysis worker...")
print(f"🔗 Connected to backend: {CONFIG['backend_url']}")
print(f"⚙️ Configuration: {CONFIG}")

while True:
    try:
        # Check for pending jobs
        response = requests.get(
            f"{CONFIG['backend_url']}/jobs",
            headers={"Authorization": f"Bearer {CONFIG['api_key']}"}
        )
        
        if response.status_code == 200:
            jobs = response.json()
            for job in jobs:
                if job['status'] == 'pending':
                    print(f"🔍 Processing job {job['id']}")
                    result = process_video(job['id'], job['video_url'])
                    
                    # Update backend
                    requests.put(
                        f"{CONFIG['backend_url']}/jobs/{job['id']",
                        json=result,
                        headers={"Authorization": f"Bearer {CONFIG['api_key']}"}
                    )
                    print(f"✅ Completed job {job['id']}")

        clear_output(wait=True)
        print(f"⏳ Next check in {CONFIG['poll_interval']}s...")
        time.sleep(CONFIG['poll_interval'])
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        time.sleep(60)